# DLO-JZ Optimisation de l'apprentissage - Jour 1 

Optimisation système d'une boucle d'apprentissage *Resnet-50*.

![car](./images/optimisation.png)

## Objet du notebook

Le but de ce *notebook* est d'optimiser un code d'apprentissage d'un modèle *Resnet-50* sur *Imagenet* pour Jean Zay en implémentant :
* **TP 1** : l'accélération GPU
* **TP 2** : l'*Automatic Mixed Precision*
* **TP 3** : le *Channels Last Memory Format*

Les cellules dans ce *notebook* ne sont pas prévues pour être modifiées, sauf rares exceptions indiquées dans les commentaires. Les TP se feront en modifiant les codes `dlojz1_X.py`.

Les directives de modification seront marquées par l'étiquette **TODO** dans le *notebook* suivant.
 
Les solutions sont présentes dans le répertoire `solutions/`.

*Notebook rédigé par l'équipe assistance IA de l'IDRIS, juin 2023*

------------------------

### Environnement de calcul

Un module PyTorch doit avoir été chargé pour le bon fonctionnement de ce Notebook. **Nécessairement**, le module `pytorch-gpu/py3/2.1.1` :

In [ ]:
!module list

Les fonctions *python* de gestion de queue SLURM dévelopées par l'IDRIS et les fonctions dédiées à la formation DLO-JZ sont à importer.

Le module d'environnement pour les *jobs* et la taille des images sont fixés pour ce *notebook*.

**TODO :** choisir un *pseudonyme* (maximum 5 caractères) pour vous différencier dans la queue SLURM et dans les outils collaboratifs pendant la formation et la compétition.

In [ ]:
from idr_pytools import display_slurm_queue, gpu_jobs_submitter, search_log
from dlojz_tools import controle_technique, compare, GPU_underthehood, plot_accuracy, lrfind_plot, imagenet_starter
MODULE = 'pytorch-gpu/py3/2.1.1'
image_size = 224
account = 'for@a100'
name = 'pseudo'   ## Pseudonyme à choisir

assert name != 'pseudo' and name != '', 'please choose a pseudo'

Création d'un répertoire `checkpoints/` si cela n'a pas déjà été fait.

In [ ]:
!mkdir -p checkpoints

------------------------------------

### Gestion de la queue SLURM

Pour afficher vos jobs dans la queue SLURM :

In [ ]:
display_slurm_queue(name)

**Remarque**: Cette fonction est utilisée plusieurs fois dans ce *notebook*. Elle permet d'afficher la queue de manière dynamique, rafraichie toutes les 5 secondes. Elle ne s'arrête que lorsque la queue est vide. Si vous désirez reprendre la main sur le *notebook*, il vous suffira d'arrêter manuellement la cellule avec le bouton *stop*. Cela a bien sûr aucun impact les jobs soumis.

Si vous voulez retirer TOUS vos jobs de la queue SLURM, décommenter et exécuter la cellule suivante :


In [ ]:
#!scancel -u $USER

Si vous voulez retirer UN de vos jobs de la queue SLURM, décommenter, compléter et exécuter la cellule suivante :

In [ ]:
#!scancel <jobid>

--------------

### Différence entre deux scripts

Pour comparer son code avec les solutions mises à disposition, la fonction suivante permet d'afficher une page HTML contenant un différentiel de fichiers texte.

In [ ]:
s1 = "./dlojz1_1.py"
s2 = "./solutions/dlojz1_1.py"
compare(s1, s2)

Voir le résultat du différentiel de fichiers sur la page suivante (attention au spoil !) :

[compare.html](compare.html)

------------------

## Dataset et modèle

Cette partie permet de visualiser les caractéristiques du *dataset* et du modèle utilisés.

### Imagenet

#### Train set

In [ ]:
import os
import torchvision
import torchvision.transforms as transforms
import torch
import numpy as np
import matplotlib.pyplot as plt

transform = transforms.Compose([ 
        transforms.RandomResizedCrop(224),             # Random resize - Data Augmentation
        transforms.RandomHorizontalFlip(),              # Horizontal Flip - Data Augmentation
        transforms.ToTensor(),                          # convert the PIL Image to a tensor
        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                             std=(0.229, 0.224, 0.225))
        ])
    
    
train_dataset = torchvision.datasets.ImageNet(root=os.environ['ALL_CCFRSCRATCH']+'/imagenet',
                                                  transform=transform)
train_dataset

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,    
                                           batch_size=4,
                                           shuffle=True)
batch = next(iter(train_loader))
print('X train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[0].shape, batch[0].dtype, batch[0].element_size()*batch[0].nelement()))
print('Y train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[1].shape, batch[1].dtype, batch[1].element_size()*batch[1].nelement()))

img = batch[0][0].numpy().transpose((1,2,0))
plt.imshow(img)
plt.axis('off')
labels_cls, labels_id = torch.load(os.environ['ALL_CCFRSCRATCH']+'/imagenet/meta.bin')
label = labels_cls[np.unique(labels_id)[batch[1][0].numpy()]]
_ = plt.title('label class: {}'.format(label[0]))

#### Validation set

In [ ]:
val_transform = transforms.Compose([
                                    transforms.Resize((256, 256)),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),   # convert the PIL Image to a tensor
                                    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                    std=(0.229, 0.224, 0.225))])

val_dataset = torchvision.datasets.ImageNet(root=os.environ['ALL_CCFRSCRATCH']+'/imagenet', split='val',
                        transform=val_transform)
val_dataset

### Resnet-50

In [ ]:
import torchvision.models as models
model = models.resnet50()
print('number of total parameters: {}'.format(sum([p.numel() for p in model.parameters()])))
print('number of trainable parameters: {}'.format(sum([p.numel() for p in model.parameters() if p.requires_grad])))

--------------


# TP1_0 : Baseline CPU

Ce TP consiste à appliquer le code *baseline* pour prendre en main les fonctionnalités de test et découvrir le code.

**TODO :**
1. Exécuter les cellules suivantes (le *job* prend plus de 10 minutes environ)
2. Puis, ouvrir le fichier [dlojz1_0.py](dlojz1_0.py)

Remarque :
* l'option *test* lance un apprentissage de 50 itérations.
* les chronomètres mesurent les temps de la 2e à la 50e itération et restitue un temps moyen par itération.
* les parties `DON'T MODIFY` dans le script ne doivent pas être modifiées.

In [ ]:
n_gpu = 1
batch_size = 128

In [ ]:
command = f'./dlojz1_0.py -b {batch_size} --image-size {image_size} --test --no-pin-memory'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['2107821']

In [ ]:
display_slurm_queue(name)

### Quizz

L'éxécution étant assez longue (> 10 min.), un quizz vous attend : [Quizz TP1_0](https://www.deepmama.com/quizz/dlojz_quizz1.html)

In [ ]:
controle_technique(jobid)

Le code *baseline* `dlojz1_0.py` a été exécuté sur le CPU (contrairement à ce qui est indiqué par le contrôle technique) en mode *test*, soit sur 50 itérations.

Dans le prochain exercice nous verrons ensemble l'accélération sur 1 GPU.


![Garage](images/stop.png "Arrêtez-vous ici! Une présentation vous attend avant le prochain TP.")

--------------------------------

# TP1_1 : Accélération GPU

Voir la [documentation pytorch](https://pytorch.org/docs/stable/generated/torch.Tensor.to.html)

**TODO** : dans le script [dlojz1_1.py](./dlojz1_1.py):
* Définir la variable `gpu` et envoyer le modèle dans la mémoire du GPU.

* Envoyer les *batches* d'images d'entrée et les *labels* associés sur le GPU, pour **les étapes de *training* et de *validation***.

In [ ]:
n_gpu = 1
batch_size = 128
command = f'./dlojz1_1.py -b {batch_size} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['902342']

In [ ]:
display_slurm_queue(name)

### Quizz

L'éxécution étant assez longue, un quizz vous attend : [Quizz TP1_1](https://www.deepmama.com/quizz/dlojz_quizz2.html)

In [ ]:
controle_technique(jobid)

### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobids = ['902357', '902358', '902359', '902360', '902361', '902362', '902363', '902365']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

Le dernier *job* a atteint le seuil *CUDA Out Of Memory* :

In [ ]:
controle_technique([jobids[-1]])

![Garage](images/stop.png "Arrêtez-vous ici! Une présentation vous attend avant le prochain TP.")

-------------------------------------

# TP1_2 : Automatic Mixed Precision

Voir la [documentation de l'IDRIS](http://www.idris.fr/ia/mixed-precision.html#en_pytorch)

**TODO** : dans le script [dlojz1_2.py](./dlojz1_2.py):
* Importer les fonctionnalités liées à l'*Automatic Mixed Precision*.

* Initialiser le *scaler*.  

* Implémenter l'*autocasting* (le changement de précision, FP32 à FP16) dans le *forward* , avec la ligne `with autocast():` dans la boucle de *training* **et** la boucle de validation.

* Implémenter le *gradient scaling* pour la seule boucle de *training*. **Note**: À la place des lignes `loss.backward()` et `optimizer.step()`.


In [ ]:
n_gpu = 1
batch_size = 128
command = f'./dlojz1_2.py -b {batch_size} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['902431']

In [ ]:
display_slurm_queue(name)

### Quizz

L'éxécution étant assez longue, un quizz vous attend : [Quizz TP1_2](https://www.deepmama.com/quizz/dlojz_quizz3.html)

In [ ]:
controle_technique(jobid)

### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobids = ['903148', '903150', '903151', '903152', '903154', '903155', '903156', '903157']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

### Changement de taille de batch

**TODO :** Choisir pour la suite du TP une taille de batch par GPU qui vous semble la plus pertinente selon le test précédent.

In [ ]:
## Choisir un batch size optimal
bs_optim = 512

In [ ]:
n_gpu = 1
command = f'./dlojz1_2.py -b {bs_optim} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['903167']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

![Commentaires](images/cedez.png "Assurez-vous que tout se passe bien avant de continuer!")

-----------------------------------------

# TP1_3 : Channels Last Memory Format

Voir la [documentation pytorch](https://pytorch.org/tutorials/intermediate/memory_format_tutorial.html)

**TODO** : dans le script [dlojz1_3.py](./dlojz1_3.py):
* Lors de l'envoie du modèle au GPU, configurer le paramètre `memory_format` avec l'option *Channel Last Memory*.

* Lors de l'envoie des images d'entrée au GPU, configurer le paramètre `memory_format` avec l'option *Channel Last Memory*.


In [ ]:
n_gpu = 1
command = f'./dlojz1_3.py -b {bs_optim} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['902659']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobids = ['902829', '902830', '902831', '902832', '902833', '902834', '902835', '902836']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

### Optionnel:
Voir la [documentation pytorch](https://pytorch.org/docs/stable/generated/torch.Tensor.to.html)

**TODO** : dans le script [dlojz1_3.py](./dlojz1_3.py):
* Configurer lors de l'envoi des images et des labels vers le *GPU*, le paramètre `non_blocking=args.non_blocking`, afin d'activer l'option *non-blocking*. Cette option sera expliqué plus tard lors de la définition d'un DataLoader distribué.

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobids = ['902813', '902814', '902816', '902817', '902819', '902820', '902821', '902823']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

![Garage](images/stop.png "Arrêtez-vous ici! Une présentation vous attend avant le prochain TP.")

----------------------